In [1]:
# !pip install prettytable

In [2]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)


from ssod.apis import get_root_logger, set_random_seed, train_detector
from ssod.datasets import build_dataset
from ssod.utils import patch_config

# from ssod.datasets.dataset_wrappers import *

/opt/ml/detection/baseline/mmdetection/mmdet/datasets/pipelines/formating.py:7: UserWarning: DeprecationWarning: mmdet.datasets.pipelines.formating will be deprecated, please replace it with mmdet.datasets.pipelines.formatting.
  warnings.warn('DeprecationWarning: mmdet.datasets.pipelines.formating will be '


In [3]:
# classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
#            "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# cfg = Config.fromfile('./2ed/85_cascade_swins_fpn_3x_iou_change.py')
# cfg.work_dir = './work_dirs/85_cascade_swins_fpn_3x_iou_change'
# root='../../dataset/'
# cfg.log_config.hooks[1].init_kwargs.name = 'MM_2nd_reappearence'

In [4]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기

# cfg = Config.fromfile('./configs/swin/retinanet_swin-t-p4-w7_fpn_1x_coco.py')
# cfg = Config.fromfile('./77_cascade_swins_fpn_3x_TTA.py')
# cfg = Config.fromfile('./2ed/85_cascade_swins_fpn_3x_iou_change.py')
cfg = Config.fromfile('../SoftTeacher/configs/soft_teacher/soft_teacher_faster_rcnn_r50_caffe_fpn_coco_full_720k.py')

root='../../dataset/'
# print(cfg.model.keys())
# dataset config 수정
# cfg.data.train.classes = classes
# cfg.data.train.img_prefix = root
# cfg.data.train.ann_file = root + 'train.json' # train json 정보
# cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

##soft teacher
cfg.data.train.sup.img_prefix = root
cfg.data.train.sup.ann_file = root + 'train.json' # train json 정보
cfg.data.train.unsup.img_prefix = root
cfg.data.train.unsup.ann_file = root + 'test.json' # train json 정보


cfg.data.sampler.train = dict(type="GroupSampler")  


cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
# cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2022
cfg.gpu_ids = [0]
# cfg.work_dir = './work_dirs/2_cascade_swins_fpn_adamW_resize1024'
# cfg.work_dir = './work_dirs/3_MM_nas_fcos_swins_adamW'
# cfg.work_dir = './work_dirs/85_cascade_swins_fpn_3x_iou_change'
cfg.work_dir = './work_dirs/3_soft_teacher_base_720'
# cfg.work_dir = './work_dirs/77_cascade_swins_fpn_3x_TTA_2'

cfg.model.roi_head.bbox_head.num_classes = 10

# cfg.model.roi_head.bbox_head[0].num_classes = 10
# cfg.model.roi_head.bbox_head[1].num_classes = 10
# cfg.model.roi_head.bbox_head[2].num_classes = 10

# cfg.model.bbox_head.num_classes = 10



cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)

# cfg.log_config.hooks[1].init_kwargs.name = 'MM_cas_rcn_swins_adamW_resize1024'
# cfg.log_config.hooks[1].init_kwargs.name = 'MM_nas_fcos_swins_adamW'
# cfg.log_config.hooks[1].init_kwargs.name = 'MM_2nd_reappearence'
cfg.log_config.hooks[1].init_kwargs.name = 'MM_3_soft_teacher_base_720_res50'




In [5]:
#  error in f16 : argument of type 'NoneType' is not iterable
# cfg.runner.meta = dict(fp16 = dict(loss_scaler=["dynamic"])) #https://github.com/open-mmlab/mmdetection/issues/5611#issuecomment-880262904
# but TypeError: list indices must be integers or slices, not str
# cfg.runner.meta = dict(fp16 = dict(loss_scaler=dict(scale="dynamic")))








# but AssertionError: __   at [assert 'policy' in lr_config] in base_runner.py 
# config/soft_teacher/base.py 의 fp16 변수를 주석처리
# but AssertionError: __   at [assert hasattr(model, "teacher")] in ssod/utils/hooks/mean_teacher.py


In [6]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.21s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [7]:
#KeyError: 'SemiDataset is not in the dataset registry'
#https://github.com/open-mmlab/mmdetection/issues/3751
#datasets/__init__.py 수정

In [8]:
# dataset 확인
datasets[0]

In [9]:
import warnings

# 경고 메시지를 무시하고 숨기거나
# warnings.filterwarnings(action='ignore')

# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()


# warnings.filterwarnings(action='default')
#https://computer-science-student.tistory.com/275

2022-03-29 06:26:51,138 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'open-mmlab://detectron2/resnet50_caffe'}
2022-03-29 06:26:51,140 - mmcv - INFO - load model from: open-mmlab://detectron2/resnet50_caffe
2022-03-29 06:26:51,141 - mmcv - INFO - load checkpoint from openmmlab path: open-mmlab://detectron2/resnet50_caffe
2022-03-29 06:26:51,216 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: conv1.bias

2022-03-29 06:26:51,240 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2022-03-29 06:26:51,267 - mmcv - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2022-03-29 06:26:51,276 - mmcv - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier',

In [10]:
# 모델 학습

train_detector(model, datasets[0], cfg, distributed=False, validate=False)

2022-03-29 06:26:54,225 - mmdet.ssod - INFO - Start running, host: root@71b0d7392d91, work_dir: /opt/ml/detection/baseline/SoftTeacher/work_dirs/3_soft_teacher_base_720
2022-03-29 06:26:54,226 - mmdet.ssod - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(ABOVE_NORMAL) Fp16OptimizerHook                  
(NORMAL      ) CheckpointHook                     
(NORMAL      ) WeightSummary                      
(NORMAL      ) MeanTeacher                        
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterH

--data_loaders--
--cfg.workflow--
[('train', 1)]


TypeError: argument of type 'NoneType' is not iterable

In [ ]:
# cfg.momentum_config